In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
from spiral_galaxy import SpiralGalaxy

[INFO]: No release version set. Setting default to MPL-11
[WARNING]: path /home/sshamsi_haverford_edu/sas/mangawork/manga/spectro/analysis/v3_1_1/3.1.0/dapall-v3_1_1-3.1.0.fits cannot be found. Setting dapall to None. (MarvinUserWarning)


In [3]:
plt.style.use('seaborn')
plt.style.use('seaborn-colorblind')

In [5]:
usable_paths = np.load('usable_v3_paths.npy')

In [6]:
gal_objs = [SpiralGalaxy(path) for path in usable_paths]

FileNotFoundError: [Errno 2] No such file or directory: '/raid5/homes/sshamsi/sas/mangawork/manga/sandbox/galaxyzoo3d/v3_0_0/1-10166_91_14709872.fits.gz'

In [ ]:
mangaid_list = [gal.mangaid for gal in gal_objs]

In [ ]:
for gal in gal_objs:
    gal.form_global_df()

In [ ]:
for gal in gal_objs:
    gal.df['Mass'] = gal.mass
    
for gal in gal_objs:
    gal.df['$\lg{Mass}$'] = gal.lgmass
    
for gal in gal_objs:
    gal.df['MaNGA ID'] = gal.mangaid

In [ ]:
for gal in gal_objs:
    map_ha = gal.maps.getMap('emline_gflux', channel='ha_6564')
    map_hb = gal.maps.getMap('emline_gflux', channel='hb_4862')

    ha_snr = map_ha.snr.flatten()
    hb_snr = map_hb.snr.flatten()

    gal.df['S/N $H_{\\alpha}$'] = ha_snr
    gal.df['S/N $H_{\\beta}$'] = hb_snr

In [ ]:
df_list = [gal.df for gal in gal_objs]

spax_df = pd.concat(df_list)

In [ ]:
spax_df = spax_df.dropna()
spax_df = spax_df[(spax_df.Comp == 0) & (spax_df.AGN == 0) & (spax_df.Seyfert == 0) & (spax_df.Liner == 0)]
spax_df = spax_df[(spax_df['$r/r_e$'] <= 1) & (spax_df['$r/r_e$'] >= 0.1)]
spax_df = spax_df[(spax_df['S/N $H_{\\alpha}$'] >= 3) & (spax_df['S/N $H_{\\beta}$'] >= 3)]

In [ ]:
radial_bins = np.linspace(0.1, 1, 5)

radial_labels = []

for i in range(len(radial_bins) - 1):
    radial_labels.append((radial_bins[i] + radial_bins[i + 1]) / 2)

In [ ]:
spax_df['Radial Bin'] = pd.cut(spax_df['$r/r_e$'], bins=radial_bins, labels=radial_labels)

In [ ]:
cut, mass_bins = pd.qcut(spax_df['$\lg{Mass}$'], q=3, retbins=True)

In [ ]:
mass_labels = [round((mass_bins[i] + mass_bins[i + 1]) / 2, 2) for i in range(3)]

In [ ]:
spax_df['Mass Bin'] = pd.qcut(spax_df['$\lg{Mass}$'], q=3, labels=mass_labels)

In [ ]:
def avg_relative_sfr(main_df, mangaid_list, gal_objs):
    ncoor_sfr_spiral, ncoor_sfr_var_spiral = [], []
    ncoor_sfr_nspiral, ncoor_sfr_var_nspiral = [], []
    
    unique_ids = main_df['MaNGA ID'].unique()
    
    tot_n_spiral, tot_n_nspiral = 0, 0
    
    for mangaid in unique_ids:
        df = main_df[main_df['MaNGA ID'] == mangaid]
        df_spiral = df[df['Spiral Arm'] == 1]
        df_nspiral = df[df['Spiral Arm'] == 0]
        
        n_spiral = len(df_spiral.index)
        n_nspiral = len(df_nspiral.index)
        
        index = mangaid_list.index(mangaid)
            
        if n_spiral > 0:
            sfr_spiral, sfr_spiral_stdv = gal_objs[index].get_sfr(df_spiral.index, avg=True)
            
            ncoor_sfr_spiral.append(sfr_spiral * n_spiral)
            ncoor_sfr_var_spiral.append((sfr_spiral_stdv**2) * n_spiral)
            
            tot_n_spiral += n_spiral
        
        if n_nspiral > 0:
            sfr_nspiral, sfr_nspiral_stdv = gal_objs[index].get_sfr(df_nspiral.index, avg=True)
            
            ncoor_sfr_nspiral.append(sfr_nspiral * n_nspiral)
            ncoor_sfr_var_nspiral.append((sfr_nspiral_stdv**2) * n_nspiral)
            
            tot_n_nspiral += n_nspiral
    
    avg_sfr_spiral = sum(ncoor_sfr_spiral) / tot_n_spiral
    avg_sfr_stdv_spiral = np.sqrt(sum(ncoor_sfr_var_spiral) / tot_n_spiral)
    
    avg_sfr_nspiral = sum(ncoor_sfr_nspiral) / tot_n_nspiral
    avg_sfr_stdv_nspiral = np.sqrt(sum(ncoor_sfr_var_nspiral) / tot_n_nspiral)
    
    rel_sfr = (avg_sfr_spiral - avg_sfr_nspiral) / avg_sfr_nspiral
    
    rel_sfr_stdv = rel_sfr * np.sqrt(((avg_sfr_stdv_spiral**2 + avg_sfr_stdv_nspiral**2) / (avg_sfr_spiral - avg_sfr_nspiral)**2)
                                     + (avg_sfr_stdv_nspiral / avg_sfr_nspiral)**2)
    
    return rel_sfr, rel_sfr_stdv

In [ ]:
spax_df1 = spax_df[spax_df['Mass Bin'] == mass_labels[0]]
spax_df2 = spax_df[spax_df['Mass Bin'] == mass_labels[1]]
spax_df3 = spax_df[spax_df['Mass Bin'] == mass_labels[2]]

In [ ]:
radial_rel_sfr = []
radial_rel_sfr_stdv = []

for radial_label in radial_labels:
    df = spax_df1[spax_df1['Radial Bin'] == radial_label]
    vals = avg_relative_sfr(df, mangaid_list, gal_objs)
    
    radial_rel_sfr.append(vals[0])
    radial_rel_sfr_stdv.append(vals[1])

In [ ]:
radial_rel_sfr2 = []
radial_rel_sfr_stdv2 = []

for radial_label in radial_labels:
    df = spax_df2[spax_df2['Radial Bin'] == radial_label]
    vals = avg_relative_sfr(df, mangaid_list, gal_objs)
    
    radial_rel_sfr2.append(vals[0])
    radial_rel_sfr_stdv2.append(vals[1])

In [ ]:
radial_rel_sfr3 = []
radial_rel_sfr_stdv3 = []

for radial_label in radial_labels:
    df = spax_df3[spax_df3['Radial Bin'] == radial_label]
    vals = avg_relative_sfr(df, mangaid_list, gal_objs)
    
    radial_rel_sfr3.append(vals[0])
    radial_rel_sfr_stdv3.append(vals[1])

In [ ]:
plt.figure(figsize=(12, 9))

plt.errorbar(radial_labels, radial_rel_sfr, yerr=radial_rel_sfr_stdv, label='Mass 9.55 $lg(M_*/M_☉)$', capsize=20,
             capthick=3)
plt.errorbar(radial_labels, radial_rel_sfr2, yerr=radial_rel_sfr_stdv2, label='Mass 10.34 $lg(M_*/M_☉)$', capsize=15,
             capthick=6)
plt.errorbar(radial_labels, radial_rel_sfr3, yerr=radial_rel_sfr_stdv3, label='Mass 10.97 $lg(M_*/M_☉)$', capsize=10,
             capthick=9)

plt.xlabel('$r/r_e$', labelpad=20, size=20)
plt.ylabel(r'Average $\frac{SFR_{spiral} - SFR_{non-spiral}}{SFR_{non-spiral}}$', labelpad=20, size=25)
plt.xticks(size=20)
plt.yticks(size=20)

plt.legend(fontsize=20)

In [ ]:
mass_labels